# Data QA Engineer Assessment - Question 3

In [4]:
# Imports

import pandas as pd
import numpy as np

#### Question 3-1

We need to review this file for overall data quality and highlight any potential problems so that they can be discussed with the data provider. Write a series of tests that will identify any errors in the file and provide a list of all errors found. Do not consider any medical details of the treatments, this question does not require any healthcare knowledge. These quality checks should include, but are not limited to:
- Duplicates
- Missing Values
- All standardized codes are valid based on given reference
material
- Date values are logical and chronological trending is
consistent

In [5]:
# Functions
def Unique_Values(lst):
   result = []
   for item in lst: 
      if item not in result: 
         result.append(item) 
   return result


In [6]:
########################################################################################
# Setup Report
########################################################################################

# Load Dataframe
df_claims = pd.read_csv("data\\sample_claims.csv", na_filter=False)
df_codes_icd10 = pd.read_csv('data\\valid_icd_10_codes.csv', na_filter=False)
df_codes_cpt = pd.read_csv('data\\valid_cpt_codes.csv', na_filter=False)

# Define Fields
date_fields = ["date_service", "date_received"]
unique_fields = ["claim_id"]
nonnullable_fields = ["patient_id","claim_id","procedure_code","date_service","date_received"]
compare_date_fields = ["date_service", "date_received"]

invalid_records = {
   "total_records": 0,
   "invalid_dates_count": 0,
   "duplicate_data_count": 0,
   "out_of_order_count": 0,
   "null_data_count": 0,
   "invalid_diagosis_count" : 0,
   "invalid_procedure_count" : 0,
}

# Count Total Records
invalid_records["total_records"] = len(df_claims)

########################################################################################
# Assessment Question Code Responses
########################################################################################

# Question 3-1, Find Duplicate Unique Fields
if len(unique_fields) > 0:
   for field in unique_fields:
      results = df_claims[df_claims.duplicated([field], keep=False)]
      invalid_records["duplicate_data_count"] = invalid_records["duplicate_data_count"] + len(results)
      
# Question 3-2, Find Null Nonnullable Fields
if len(nonnullable_fields) > 0:      
   for field in nonnullable_fields:
      results = df_claims[df_claims[field].isnull() | (df_claims[field] == "")]
      invalid_records["null_data_count"] = invalid_records["null_data_count"] + len(results)
   
# Question 3-3 (part 1), Find Invalid Diagnosis Codes
df_merged = (
   df_claims
      .assign(row_number=range(len(df_claims)))
      .assign(diagnosis_codes=df_claims['diagnosis_codes'].str.split('^'))
      .explode('diagnosis_codes')
      .merge(df_codes_icd10, left_on='diagnosis_codes', right_on='code', how='left', indicator=True)
)
results = df_merged[df_merged['code'].isnull()]
distinct_records = Unique_Values(results['row_number'].tolist())
invalid_records["invalid_diagosis_count"] = len(distinct_records)

# Question 3-3 (part 2),Find Invalid Procedure Codes
df_codes_cpt['code'] = df_codes_cpt['code'].astype('string')
df_merged = (
   df_claims
      .assign(row_number=range(len(df_claims)))
      .merge(df_codes_cpt, left_on='procedure_code', right_on='code', how='left', indicator=True))
results = df_merged[df_merged['code'].isnull()]
invalid_records["invalid_procedure_count"] = len(results)

# Question 3-4 (part 1), Find Invalid Dates
if len(date_fields) > 0:
   for field in date_fields:   
      df_claims[field] = pd.to_datetime(df_claims[field], errors='coerce')
      results = df_claims[df_claims[field].isnull()]
      invalid_records["invalid_dates_count"] = invalid_records["invalid_dates_count"] + len(results)

# Question 3-4 (part 2), Find Claims Received Before Service Date
from_date = compare_date_fields[0]
to_date = compare_date_fields[1]
if len(compare_date_fields) == 2:
   df_results = df_claims[(df_claims[from_date].isnull() == False) | (df_claims[to_date].isnull() == False)]
   df_results["out_of_order"] = (df_results[to_date] < df_results[from_date])
   results = df_results[df_results["out_of_order"] == True]['claim_id']
   invalid_records["out_of_order_count"] = invalid_records["out_of_order_count"] + len(results)

########################################################################################
# Report Output
########################################################################################

# Generate Report:
print(f"""
      There are {invalid_records["total_records"]} total records.
            
      Data Integrity Checks, Question 3-1, 3-2, and 3-4
      ----------------------------------------
      - Question 3-1 (Duplicates) 
      - Found {invalid_records["duplicate_data_count"]} occurrences of duplicate values in unique fields.
      
      - Question 3-2 (Missing Values) 
      - Found {invalid_records["null_data_count"]} occurrences of null data in a nonnullable field.
      
      - Question 3-4 (Date Values Logical and Chronological) 
      - Found {invalid_records["invalid_dates_count"]} occurrences of invalid dates that resulted in a NULL value.
      - Found {invalid_records["out_of_order_count"]} occurrences claims that were serviced after the receive date.
            
      Data Quality Checks, Question 3-3
      ----------------------------------------
      - Found {invalid_records["invalid_diagosis_count"]} records referencing at least one diagnosis codes in the collection that does not exist in the provided extracts.
      - This represents {invalid_records["invalid_diagosis_count"]/invalid_records["total_records"] * 100}% of all records
      
      - Found {invalid_records["invalid_procedure_count"]} records referencing cpt code that does not exist in the provided extracts.
      - This represents {invalid_records["invalid_procedure_count"]/invalid_records["total_records"] * 100}% of all records
      """)


      There are 5000 total records.
            
      Data Integrity Checks, Question 3-1, 3-2, and 3-4
      ----------------------------------------
      - Question 3-1 (Duplicates) 
      - Found 56 occurrences of duplicate values in unique fields.
      
      - Question 3-2 (Missing Values) 
      - Found 21 occurrences of null data in a nonnullable field.
      
      - Question 3-4 (Date Values Logical and Chronological) 
      - Found 20 occurrences of invalid dates that resulted in a NULL value.
      - Found 81 occurrences claims that were serviced after the receive date.
            
      Data Quality Checks, Question 3-3
      ----------------------------------------
      - Found 4956 records referencing at least one diagnosis codes in the collection that does not exist in the provided extracts.
      - This represents 99.11999999999999% of all records
      
      - Found 1696 records referencing cpt code that does not exist in the provided extracts.
      - This repr